# Analise dos dados da tabela

## Fazendo os imports necessários

In [178]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from database.db_connection import DatabaseConnection
from database.Repositories.AppointedHours_repository import AppointedHours_repository
from database.Repositories.ProjectPlanning_repository import ProjectPlanning_repository

<p> Transformando os registros do banco em dataFrames <p>

In [182]:
database = DatabaseConnection()
database.connect()

appointed_repository = AppointedHours_repository(database)
planning_repository = ProjectPlanning_repository(database)

appointments_df = pd.DataFrame(appointed_repository.get_all_appointed_hours())
plannings_df = pd.DataFrame(planning_repository.get_all_pplaning())

database.close()

app_df = appointments_df.copy()
ppl_df = plannings_df.copy()


<p> Analisando as informações de apontamentos <p>

In [183]:
app_df

,_id,user,project,entry_id,start_at,end_at,worked_hours
0,67d879553e5437078dbdc23e,66bb5bd942e1a97ae88b07cd,66bb5c27ae6f2c5756808038,6776ef280f50991d14159f64,2025-01-01 21:30:00,2025-01-02 03:20:00,5.83
1,67d879553e5437078dbdc23f,66a8e1a6248c237b82970d2d,63247e8515e8e969ad22ec43,6776fd3d989ab45f82d80fb9,2025-01-01 16:00:00,2025-01-01 17:30:00,1.50
2,67d879553e5437078dbdc240,6352d69a9cc2917e2fdcffa9,677d27b1c22bea50e61c512d,677d2a2f1a650a30078f5265,2025-01-01 16:00:00,2025-01-01 21:00:00,5.00
3,67d879553e5437078dbdc241,66d73a7b5fbae338800e08d6,67054bd5d2f2b20f74681c89,67977be6bc81e53f1a6f503e,2025-01-01 12:00:00,2025-01-01 20:30:00,8.50
4,67d879553e5437078dbdc242,6352d69a9cc2917e2fdcffa9,677d27b1c22bea50e61c512d,677d2a212622fe7a0197f8d3,2025-01-01 11:00:00,2025-01-01 15:00:00,4.00
...,...,...,...,...,...,...,...
11369,67d8837ac131bb2a9ad1e28b,6671c085842d3958e34fc70e,633f11b385ab8668de6ae8c8,67d5157ccf18252186475f66,2025-03-15 04:00:00,2025-03-15 06:00:00,2.00
11370,67d8837cc131bb2a9ad1e28c,66a8e1a6248c237b82970d2d,63247e8515e8e969ad22ec43,67d8503836c8ee4d3e03f248,2025-03-16 20:00:00,2025-03-16 20:30:00,0.50
11371,67d8837cc131bb2a9ad1e28d,66a8e1a6248c237b82970d2d,63247e8515e8e969ad22ec43,67d850326f115313978f6718,2025-03-16 17:00:00,2025-03-16 17:15:00,0.25
11372,67d8837cc131bb2a9ad1e28e,63b333604ea5ce158d21ae2e,6564a69ec57c0265f16c4531,67d82b7b7c04a4220679e67d,2025-03-16 12:00:00,2025-03-16 20:00:00,8.00


<p> Analisando as informações de planejamento de projetos <p>

In [184]:
ppl_df

,_id,planning_id,project_id,date_start,date_end,contracted_hours,used_hours,remaining_hours
0,67d879563e5437078dbdc243,288d832c-d7f5-ef11-be20-000d3a894417,678a68d23591ef251cd44583,2025-02-01,2025-02-28,10.0,0.0,10.0
1,67d879563e5437078dbdc244,4f7831f6-d7f5-ef11-be20-000d3a894417,65ba6f93dc7a9a37f56fcba0,2025-02-01,2025-02-28,20.0,0.0,13.0
2,67d879563e5437078dbdc245,2e86085f-d8f5-ef11-be20-000d3a894417,670fef69d08c20561620eff9,2025-02-01,2025-02-28,15.0,0.0,0.0
3,67d879563e5437078dbdc246,33d62f1e-8ff0-ef11-be21-00224836594b,66fc64d1abb3a15f1721b428,2025-02-01,2025-02-28,100.0,0.0,0.0
